# Reading course project
The goal of this project is to see whether we can train a regression model to predict the power usage of a house just by using the output of it's smart sensors, in particular we're going to use the datasets number 7 and 8 from Kyoto, taken from [Casas Wsu project](http://casas.wsu.edu/datasets/).
The second dataset has records from three different types of sensors:
1. Mxx, motion sensors
2. Txx, temperature sensors
3. P001, eletricity usage

While the first one has some additional ones:
1. AD1-A, burner sensor
2. AD1-B, hot water sensor
3. AD1-C, cold water sensor

We're going to try three different attempts, in the first one we're going to use only the first dataset to train the  model as it contains a wider variety of sensors, in the second one we're only going to use the second dataset, and in the third one we're going to use both the data from the first dataset and the second but only with the output of the three shared sensors. Afterwards we're going to compare the results from the three attempts to see which one gives better results.


## First attempt
We are going to use two different data sources from the first dataset, the first one is a file called **raw.txt** and it contains the output from the power usage sensor, the motion sensors and the temperature sensors, this data will be taken from a .txt file.
The second one is a Postgres database called **casaskyoto** and it contains the output from the burner sensor, hot and cold water sensors.
The database was created using the file **annotated_formatted.txt**, which is the output of the **annotated.txt** file from the dataset formatted using the **formatter.ipynb** script, which was then given in input to the script [Activity forecasting db](https://github.com/Piier/activity_forecasting_db).

The database contains four different tables:
- One for the activities

<img src="https://i.postimg.cc/3RFPQkf5/activity-table.jpg" width="900"/>


- One for the events

<img src="https://i.postimg.cc/8ckxpTxF/event-table.jpg" width="700"/>

- One for the houses

<img src="https://i.postimg.cc/L5ywhrMx/house-table.jpg" width="500"/>

- One for the sensors

<img src="https://i.postimg.cc/y6nwfSS0/sensor-table.jpg" width="700"/>

For the first file **raw.txt** we're going to create a dataframe, this dataframe will then be split into 3 different dataframes:
1. **df_p001**, which stores the data from the power sensor;
2. **df_mxx**, which stores the data from the movement sensors;
3. **df_txx**, which stores the data from the temperature sensors.

Each of these dataframe is going to be resampled using different time windows (10 minutes, 15 minutes, 30 minutes, etc.), using the parameter "res_rate", and the value for the power sensor and the temperature sensors are going to be grouped using the average, while the movement sensors data is going to be grouped using the sum of all activated movement sensors during that time window.
Afterwards we're going to join these three dataframes into a single one, using 'datetime' as the key.

### Feature creation
We're also going to add some time related features:
1. **time_of_day**, an integer from 0 to 23 representing the hour of the day for each sample;
2. **day_of_week**, an integer from 0 to 6 representing the day of the week. This could be useful in the prediction of the power usage, since usually during the weekend the usage is lower during the day and higher during the night.

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
#Creation of the dataset from the raw file
df_raw = pd.read_csv("./twor_2009/raw.txt", header = None, sep='\t', index_col = False,
                   names=["datetime", "sensor", "value"])

In [2]:
#We remove the values from the sensors in the cabinet and the door sensors
df_raw = df_raw[(df_raw['value'] != 'PRESENT') & (df_raw['value'] != 'ABSENT')
       & (df_raw['value'] != 'CLOSE') & (df_raw['value'] != 'OPEN')]

#cast datetime field as datetime type
df_raw['datetime'] = pd.to_datetime(df_raw["datetime"])
#We only keep the value that says when a sensor has been turned on
df_raw = df_raw[df_raw['value'] != 'OFF']
#Then we set it to 1, to be able to sum the number of sensors later
df_raw.replace('ON','1',regex=True, inplace = True)

In [3]:
#dataframe that only contains the data for the power meter
df_p001 = df_raw[df_raw['sensor'] == 'P001']
df_p001['value'] = pd.to_numeric(df_p001["value"])
df_p001 = df_p001[['value', 'datetime']]

In [4]:
#Definition of a variable controlling the resampling rate
res_rate = '15min'
#We resample the data every thirty minutes and then take the mean
df_p001 = df_p001.resample(res_rate, on='datetime').value.mean()
#Resample returns a series, so we have to make it into a dataframe and reset the index
df_p001 = pd.DataFrame(df_p001)
df_p001 = df_p001.rename(columns={"value": "power_consumption"})
df_p001.reset_index(inplace=True)

In [5]:
#Feature creation, we add two new columns: one for the time of the day and the other one for the day of the week
df_p001['time_of_day'] = df_p001['datetime'].apply(lambda row : row.hour)
df_p001['day_of_week'] = df_p001['datetime'].apply(lambda row : row.dayofweek)

In [6]:
df_p001.head()

,datetime,power_consumption,time_of_day,day_of_week
0,2009-02-01 01:00:00,2658.650000,1,6
1,2009-02-01 01:15:00,3420.600000,1,6
2,2009-02-01 01:30:00,2329.609677,1,6
3,2009-02-01 01:45:00,3831.529412,1,6
4,2009-02-01 02:00:00,3192.333333,2,6


In [7]:
#Dataframe for the value of the motion sensors
df_mxx = df_raw[df_raw['sensor'].str.match('^M[0-9]*')== True]
df_mxx = df_mxx[['value', 'datetime']]
df_mxx['value'] = pd.to_numeric(df_mxx["value"])

#We resample the data every hour and then take the sumy
df_mxx = df_mxx.resample(res_rate, on='datetime').value.sum()
#Resample returns a series, so we have to make it into a dataframe and reset the index
df_mxx = pd.DataFrame(df_mxx)
df_mxx.reset_index(inplace=True)

In [8]:
#Renaming of the fields to more understandable names
df_mxx = df_mxx.rename(columns={"value": "num_sensors"})

In [9]:
df_mxx.head()

,datetime,num_sensors
0,2009-02-01 07:15:00,1
1,2009-02-01 07:30:00,9
2,2009-02-01 07:45:00,6
3,2009-02-01 08:00:00,100
4,2009-02-01 08:15:00,5


In [10]:
#We merge the df for the power consumption with the one that has the number of motion sensors activated
df_merged = pd.merge(df_p001, df_mxx, on='datetime', how='left')
df_merged.dropna(inplace=True)
df_merged

,datetime,power_consumption,time_of_day,day_of_week,num_sensors
25,2009-02-01 07:15:00,2416.050000,7,6,1.0
26,2009-02-01 07:30:00,2900.090909,7,6,9.0
27,2009-02-01 07:45:00,2450.209524,7,6,6.0
28,2009-02-01 08:00:00,3201.093750,8,6,100.0
29,2009-02-01 08:15:00,2763.577273,8,6,5.0
...,...,...,...,...,...
8730,2009-05-02 23:30:00,586.371429,23,5,3.0
8731,2009-05-02 23:45:00,1245.280000,23,5,2.0
8732,2009-05-03 00:00:00,1640.068182,0,6,146.0
8733,2009-05-03 00:15:00,3181.217391,0,6,58.0


In [11]:
#Dataframe for the temperature
df_txx = df_raw[df_raw['sensor'].str.match('^T[0-9]*') == True]
#Cast the temperature value to a numeric type
df_txx['value'] = pd.to_numeric(df_txx["value"])
#Renaming on the value column
df_txx = df_txx.rename(columns={"value": "temperature"})

#We resample the data every hour and then take the mean
df_txx = df_txx.resample(res_rate, on='datetime').temperature.mean()
#Resample returns a series, so we have to make it into a dataframe and reset the index
df_txx = pd.DataFrame(df_txx)
df_txx.reset_index(inplace=True)

In [12]:
df_txx.head()

,datetime,temperature
0,2009-02-01 07:15:00,22.454545
1,2009-02-01 07:30:00,21.900000
2,2009-02-01 07:45:00,21.866667
3,2009-02-01 08:00:00,21.900000
4,2009-02-01 08:15:00,22.166667


In [13]:
#We merge the dataframe for the temperature with the previous one we created
df_merged = pd.merge(df_merged, df_txx, on='datetime', how='left')

In [14]:
df_merged.head()

,datetime,power_consumption,time_of_day,day_of_week,num_sensors,temperature
0,2009-02-01 07:15:00,2416.050000,7,6,1.0,22.454545
1,2009-02-01 07:30:00,2900.090909,7,6,9.0,21.900000
2,2009-02-01 07:45:00,2450.209524,7,6,6.0,21.866667
3,2009-02-01 08:00:00,3201.093750,8,6,100.0,21.900000
4,2009-02-01 08:15:00,2763.577273,8,6,5.0,22.166667


In [15]:
# #We plot for each day the number of movement sensors activated every hour, the average temperature and the power consumption, to see if there seems to be 
# #any correlation among those lines.
# from matplotlib.pyplot import figure
# import matplotlib
# import matplotlib.pyplot as plt
# #This is to avoid displaying the images
# plt.ioff()
# day = 1
# for i in range(17, len(df2), 24):
#     start = i
#     end = i+24
#     dfRange = df2.iloc[start:end]
#     x1 = dfRange['power_consumption'].tolist()
#     x1_div = [x / 10 for x in x1]
#     x2 = dfRange['temperature'].tolist()
#     x3 = dfRange['num_sensors'].tolist()
#     y = dfRange['datetime'].tolist()
#     fig, ax = plt.subplots()
#     fig.set_size_inches(18.5, 5, forward=True)
#     ax.plot(y, x1_div, label='PC');
#     ax.plot(y, x2, label='temperature');
#     ax.plot(y, x3, label='num_sensors');
#     plt.legend();
#     file_name = './img/power_consumption_day_' + str(day) + '.png'
#     plt.savefig(file_name);
#     plt.close(fig);
#     day += 1

### Plot analysis
From the plots we can see that there are days where the power consumption follows a similar pattern to the number of sensors activated:

<img src="https://i.ibb.co/vYVY3dv/power-consumption-day-8.png" width="900" />

And there are also days where the power consumption and the number of sensors activated don't seem to be correlated:

<img src="https://i.ibb.co/D5rZ6zZ/power-consumption-day-41.png" width="900" />

There are also some days where the number of motion sensors activated is zero for the whole day, probably meaning that the house was empty, but there is still power usage for that day:

<img src="https://i.ibb.co/82y5yQ7/power-consumption-day-103.png" width="900" />

In [16]:
#This is to take the data from the postgres db for the second file that contains data from ad1a, ad1b and ad1c
import psycopg2
from pandas import DataFrame

try:
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="casaskyoto")
    cursor = connection.cursor()
    query1= "select t1.date as datetime, t2.name as sensor, t1.value from activity.event as t1 join activity.sensor as t2 on t1.sensor = t2.id "

    cursor.execute(query1)
    data_np= DataFrame(cursor.fetchall())
    data_np.columns = [desc[0] for desc in cursor.description]
    

except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed


In [17]:
#We cast the datetime column as type datetime
data_np['datetime'] = pd.to_datetime(data_np["datetime"])

In [18]:
#We select only the rows related to those sensors using regex
data_ad1 = data_np[data_np['sensor'].str.match('^AD1-[A|B|C]')== True]

In [19]:
data_ad1.head()

,datetime,sensor,value
1255,2009-02-06 08:08:01.776720,AD1-A,2.82223
1256,2009-02-06 08:08:05.895149,AD1-A,2.79934
1257,2009-02-06 08:08:54.696549,AD1-A,2.82325
1258,2009-02-06 08:09:08.766969,AD1-A,2.79692
1259,2009-02-06 08:11:03.494780,AD1-A,2.81817


In [20]:
data_ad1.sort_values(by='datetime', inplace=True)

In [21]:
data_ad1['value'] = pd.to_numeric(data_ad1['value'])

In [22]:
#We divide into three different datasets, one for each different sensor
data_ad1a = data_ad1[data_ad1['sensor'] == 'AD1-A']
data_ad1b = data_ad1[data_ad1['sensor'] == 'AD1-B']
data_ad1c = data_ad1[data_ad1['sensor'] == 'AD1-C']

#Drop the sensor column we don't need anymore
data_ad1a.drop(['sensor'], axis = 1, inplace=True)
data_ad1b.drop(['sensor'], axis = 1, inplace=True)
data_ad1c.drop(['sensor'], axis = 1, inplace=True)

In [23]:
#We resample the data every hour and then take the sum
data_ad1a = data_ad1a.resample(res_rate, on='datetime').value.sum()
#Resample returns a series, so we have to make it into a dataframe and reset the index
data_ad1a = pd.DataFrame(data_ad1a)
data_ad1a.reset_index(inplace=True)

#We resample the data every hour and then take the sum
data_ad1b = data_ad1b.resample(res_rate, on='datetime').value.sum()
#Resample returns a series, so we have to make it into a dataframe and reset the index
data_ad1b = pd.DataFrame(data_ad1b)
data_ad1b.reset_index(inplace=True)

#We resample the data every hour and then take the sum
data_ad1c = data_ad1c.resample(res_rate, on='datetime').value.sum()
#Resample returns a series, so we have to make it into a dataframe and reset the index
data_ad1c = pd.DataFrame(data_ad1c)
data_ad1c.reset_index(inplace=True)

#Renaming of the columns for each dataset to tell them apart
data_ad1a = data_ad1a.rename(columns={"value": "sum_ad1a"})
data_ad1b = data_ad1b.rename(columns={"value": "sum_ad1b"})
data_ad1c = data_ad1c.rename(columns={"value": "sum_ad1c"})

In [24]:
df_merged = pd.merge(df_merged, data_ad1a, on='datetime', how='left')
df_merged = pd.merge(df_merged, data_ad1b, on='datetime', how='left')
df_merged = pd.merge(df_merged, data_ad1c, on='datetime', how='left')
#We fill the NaN with zeros cause it's plausible that there were some times where the burner was not used
df_merged.fillna(0, inplace=True)

In [25]:
df_merged

,datetime,power_consumption,time_of_day,day_of_week,num_sensors,temperature,sum_ad1a,sum_ad1b,sum_ad1c
0,2009-02-01 07:15:00,2416.050000,7,6,1.0,22.454545,0.0,0.0,0.0
1,2009-02-01 07:30:00,2900.090909,7,6,9.0,21.900000,0.0,0.0,0.0
2,2009-02-01 07:45:00,2450.209524,7,6,6.0,21.866667,0.0,0.0,0.0
3,2009-02-01 08:00:00,3201.093750,8,6,100.0,21.900000,0.0,0.0,0.0
4,2009-02-01 08:15:00,2763.577273,8,6,5.0,22.166667,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
8671,2009-05-02 23:30:00,586.371429,23,5,3.0,21.466667,0.0,0.0,0.0
8672,2009-05-02 23:45:00,1245.280000,23,5,2.0,21.433333,0.0,0.0,0.0
8673,2009-05-03 00:00:00,1640.068182,0,6,146.0,21.333333,0.0,0.0,0.0
8674,2009-05-03 00:15:00,3181.217391,0,6,58.0,21.366667,0.0,0.0,0.0


### Machine learning
In this part we split the dataframe into X, which will contain the values for the features that we want to use to train our regression model, and y which will be the target variable, then we split X and y into train set and test set.
Afterwards we try different ML models to see which one gives the best results. The model used are:
1. Decision tree regressor
2. Multilayer perceptron regressor
3. Linear regression
4. Support Vector Regression

In [26]:
from sklearn.model_selection import train_test_split
X = df_merged[['time_of_day', 'num_sensors','day_of_week','temperature','sum_ad1a', 'sum_ad1b', 'sum_ad1c']]
y = df_merged['power_consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
from sklearn import metrics
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_percentage_error
def print_metrics(y_true, y_pred):
    print('R2 Score:', r2_score(y_test, y_pred))
    print('MAPE :' + str(mean_absolute_percentage_error(y_test, y_pred)*100) + "%")


In [28]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
regr = DecisionTreeRegressor(max_depth=4)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.16273621347847245
MAPE :83.26052988867116%


In [29]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(random_state=1, max_iter=1000).fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.1524024406972665
MAPE :86.13785303017349%


In [30]:
from sklearn.linear_model import LinearRegression
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
y_pred = linear_regressor.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.11928295908459319
MAPE :87.74802437947089%


In [31]:
from sklearn import svm
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.07803359692196576
MAPE :82.05345552635393%


# Second try using only the second dataset
We try the same strategy, using only the second dataset.

In [32]:
#Creation of the dataset from the raw file
df_summer = pd.read_csv("./twor_2009/annotated_summer.txt", header = None, sep='\t', index_col = False,
                   names=["datetime", "sensor", "value"])

#Cast datetime field as type datetime
df_summer['datetime'] = pd.to_datetime(df_summer["datetime"])

#We get the date of the first row, will be used later to compute the timedelta
first_date = df_summer.head(1)['datetime'].values[0]

#We replace the values 'ON' for the movement sensors with one
df_summer.replace('ON','1',regex=True, inplace = True)

#We cast to numeric all the values, if a value is not a number errors='coerce' will return nan so we drop the nan
df_summer['value'] = pd.to_numeric(df_summer["value"],  errors='coerce')
df_summer.dropna(inplace=True)

In [33]:
#Create new dfs for each of the sensors we want to keep track of
df_summer_p001 = df_summer[df_summer['sensor'] == 'P001']
df_summer_mxx = df_summer[(df_summer['sensor'].str.match('^M[0-9]+')== True)]
df_summer_txx = df_summer[df_summer['sensor'].str.match('^T[0-9]+')== True]

In [34]:
# #We group by hour, we get the sum for the power and movement sensors, while we get the mean of the temperature
# df_summer_p001 = df_summer_p001.groupby('time_delta').mean()
# df_summer_mxx = df_summer_mxx.groupby('time_delta').sum()
# df_summer_txx = df_summer_txx.groupby('time_delta').mean()

#We resample the data every thirty minutes and then take the mean
df_summer_p001 = df_summer_p001.resample(res_rate, on='datetime').value.mean()
#Resample returns a series, so we have to make it into a dataframe and reset the index
df_summer_p001 = pd.DataFrame(df_summer_p001)
df_summer_p001.reset_index(inplace=True)

#We resample the data every thirty minutes and then take the mean
df_summer_mxx = df_summer_mxx.resample(res_rate, on='datetime').value.sum()
#Resample returns a series, so we have to make it into a dataframe and reset the index
df_summer_mxx = pd.DataFrame(df_summer_mxx)
df_summer_mxx.reset_index(inplace=True)

#We resample the data every thirty minutes and then take the mean
df_summer_txx = df_summer_txx.resample(res_rate, on='datetime').value.mean()
#Resample returns a series, so we have to make it into a dataframe and reset the index
df_summer_txx = pd.DataFrame(df_summer_txx)
df_summer_txx.reset_index(inplace=True)

In [35]:
df_summer_p001['time_of_day'] = df_summer_p001['datetime'].apply(lambda row : row.hour)

In [36]:
#Renaming of the fields to more understandable names
df_summer_mxx = df_summer_mxx.rename(columns={"value": "num_sensors"})
df_summer_p001 = df_summer_p001.rename(columns={"value": "power_consumption"})
df_summer_txx = df_summer_txx.rename(columns={"value": "temperature"})

In [37]:
df_merged_summer = pd.merge(df_summer_mxx, df_summer_p001, on='datetime', how='left')

In [38]:
df_merged_summer = pd.merge(df_merged_summer, df_summer_txx, on='datetime', how='left')

In [39]:
df_merged_summer = df_merged_summer.reset_index()
df_merged_summer.drop(['index'], axis = 1, inplace=True)

In [40]:
df_merged_summer.dropna(inplace=True)

In [41]:
df_merged_summer

,datetime,num_sensors,power_consumption,time_of_day,temperature
0,2009-05-29 00:15:00,6.0,3534.222222,0,24.700000
1,2009-05-29 00:30:00,3.0,1578.355556,0,24.600000
2,2009-05-29 00:45:00,4.0,496.400000,0,24.533333
3,2009-05-29 01:00:00,2.0,547.557143,1,24.400000
4,2009-05-29 01:15:00,2.0,612.440000,1,24.300000
...,...,...,...,...,...
6113,2009-07-31 16:30:00,1.0,378.854167,16,31.125000
6114,2009-07-31 16:45:00,28.0,433.260714,16,31.214286
6115,2009-07-31 17:00:00,20.0,418.265672,17,32.000000
6116,2009-07-31 17:15:00,42.0,402.905660,17,31.083333


In [42]:
X = df_merged_summer[['time_of_day', 'num_sensors','temperature']]
y = df_merged_summer['power_consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [43]:
regr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.043586119405368606
MAPE :34.76072259718001%


In [44]:
regr = MLPRegressor(random_state=1, max_iter=2000).fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.2780153991515786
MAPE :56.31789717270603%


In [45]:
regr = DecisionTreeRegressor(max_depth=4)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.3254703799312534
MAPE :54.23518755103458%


In [46]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
y_pred = linear_regressor.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.17071860856480947
MAPE :61.45657841716805%


### Third attempt
We make a third attempt using the data from the first dataset and the second one at the same time, to see if the precision of the algorithm will improve using more examples to train the model.

In [47]:
df_merged_short = df_merged.drop(['sum_ad1a','sum_ad1b', 'sum_ad1c'], axis=1)
frames = [df_merged_short, df_merged_summer]
df_concat = pd.concat(frames)

In [48]:
X = df_concat[['time_of_day', 'num_sensors','temperature']]
y = df_concat['power_consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [49]:
regr = DecisionTreeRegressor(max_depth=4)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.21731666815656847
MAPE :101.62350420873096%


In [50]:
regr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.04425999371377698
MAPE :95.6555174355628%


# Testing results and conclusions
The metrics of evaluation used for the testing of the models are:
- R2 score, the best possible value is 1.0 and can also be negative. It's computed as 1 - SSres / SStot. Where SSres is the sum of squares of the residual errors and SStot is the total sum of the errors.
- Mean absolute percentage error, computed as the subraction between the true value and the predicted value, divided by the true value and multiplied for 100. The lower the value, the better the model.

The testing was conducted using different time windows for the resampling of the dataset (10 minutes, 15 minutes, 30 minutes, 1 hour, etc.). Below we will report the results for the two most significant time windows: 15 minute and 1 hour.

<table class="tg">
    <caption>Test results (1 hour)</caption>
<thead>
  <tr>
    <th class="tg-0pky">Dataset</th>
    <th class="tg-0pky">Number of samples</th>
    <th class="tg-0pky">R2 score</th>
    <th class="tg-0pky">Mean absolute percentage error</th>
    <th class="tg-0pky">Algorithm</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">Spring 2009 (7)</td>
    <td class="tg-0pky">2177</td>
    <td class="tg-0pky">0.288</td>
    <td class="tg-0lax">48.25%</td>
    <td class="tg-0lax">Decision Tree regressor</td>
  </tr>
  <tr>
    <td class="tg-0pky">Summer 2009 (8)</td>
    <td class="tg-0pky">1354</td>
    <td class="tg-0pky">0.0451</td>
    <td class="tg-0lax">32.88%</td>
    <td class="tg-0lax">SVR</td>
  </tr>
  <tr>
    <td class="tg-0pky">Merged dataset</td>
    <td class="tg-0pky">3531</td>
    <td class="tg-0pky">0.111</td>
    <td class="tg-0lax">71.40%</td>
    <td class="tg-0lax">Decision Tree regressor</td>
  </tr>
</tbody>
</table>

From the table above we can see that the only best result in terms of MAPE comes from the summer dataset, with a value of around 32%, while the first dataset has a MAPE of around 50%.
However that MAPE value is paired with a very low R2 score, which probably tells us that the high MAPE value is probably only due to low variability in that specific dataset and not to the quality of the prediction model.
We can also notice that the increased number of samples in the third dataset, made by concatenating the first and the second dataset, did not improve the results of the prediction probably because of the diversity of the two datasets.


<table class="tg">
    <caption>Test results (15 minutes)</caption>
<thead>
  <tr>
    <th class="tg-0pky">Dataset</th>
    <th class="tg-0pky">Number of samples</th>
    <th class="tg-0pky">R2 score</th>
    <th class="tg-0pky">Mean absolute percentage error</th>
    <th class="tg-0pky">Algorithm</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">Spring 2009 (7)</td>
    <td class="tg-0pky">8676</td>
    <td class="tg-0pky">0.162</td>
    <td class="tg-0lax">83.26%</td>
    <td class="tg-0lax">Decision Tree regressor</td>
  </tr>
  <tr>
    <td class="tg-0pky">Summer 2009 (8)</td>
    <td class="tg-0pky">5047</td>
    <td class="tg-0pky">0.325</td>
    <td class="tg-0lax">54.23%</td>
    <td class="tg-0lax">Decision tree regressor</td>
  </tr>
  <tr>
    <td class="tg-0pky">Merged dataset</td>
    <td class="tg-0pky">13723</td>
    <td class="tg-0pky">0.217</td>
    <td class="tg-0lax">101.62%</td>
    <td class="tg-0lax">SVR</td>
  </tr>
</tbody>
</table>

The table above reports the results using a 15 minutes time window. We can see this change brings us better results for the second dataset, which has an increased MAPE value but a better R2 score, which tells us that we have a probably better overall prediction model, but also worse results for the other two datasets. This brings us to the conclusion that the resampling time window is a very dataset-specific parameter, and thus we can't find an ideal value but it has to be determined for that dataset.

### Prediction of future power consumption
In this part, we're going to try to predict the power consumption of the next time window, based on the data from the previous time window.
We're going to add a new column to each of the three previous dataframes based on the power consumption of the next time window, except for the last row, which doesn't have a next time window.

In [51]:
#empty list that will contain the value of the power consumption for the next row
next_pc_list = []
for index, row in df_merged.iterrows():
    next_pc = None
    #The last row won't have a next power consumption so we stop at the second to last
    if(index < len(df_merged)-1):
        next_pc = df_merged.iloc[index+1].power_consumption
    next_pc_list.append(next_pc)

In [52]:
#We create a new column for the dataframe from the list, then drop the last row that doesn't have the value
df_merged['next_pc'] = next_pc_list
df_merged.dropna(inplace=True)

In [53]:
X = df_merged[['time_of_day', 'num_sensors','day_of_week','temperature','sum_ad1a', 'sum_ad1b', 'sum_ad1c', 'power_consumption']]
y = df_merged['next_pc']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [54]:
regr = DecisionTreeRegressor(max_depth=4)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.31869887441777855
MAPE :73.82863351288862%


In [55]:
regr = MLPRegressor(random_state=1, max_iter=1000).fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.324339144294722
MAPE :76.86703336728714%


In [56]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
y_pred = linear_regressor.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.30842871930741267
MAPE :76.33409539663704%


In [57]:
regr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.26626574108451584
MAPE :75.36743522243971%


## Second dataset

In [58]:
next_pc_list = []

for index, row in df_merged_summer.iterrows():
    next_pc = None
    #The last row won't have a next power consumption so we stop at the second to last
    if(index < len(df_merged_summer)-1):
        next_pc = df_merged_summer.iloc[index+1].power_consumption
    next_pc_list.append(next_pc)
    
#We create a new column for the dataframe from the list, then drop the last row that doesn't have the value
df_merged_summer['next_pc'] = next_pc_list
df_merged_summer.dropna(inplace=True)

In [59]:
X = df_merged_summer[['time_of_day', 'num_sensors','temperature']]
y = df_merged_summer['power_consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [60]:
regr = DecisionTreeRegressor(max_depth=4)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.26850802177773647
MAPE :50.31776770302524%


In [61]:
regr = MLPRegressor(random_state=1, max_iter=3000).fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.2529017868784801
MAPE :53.45900122257022%


In [62]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
y_pred = linear_regressor.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.16075225894473877
MAPE :57.42736564256871%


In [63]:
regr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.0029953517249381445
MAPE :32.389458630507036%


## Third dataset

In [64]:
next_pc_list = []
for index, row in df_concat.iterrows():
    next_pc = None
    #The last row won't have a next power consumption so we stop at the second to last
    if(index < len(df_concat)-1):
        next_pc = df_concat.iloc[index+1].power_consumption
    next_pc_list.append(next_pc)
    
#We create a new column for the dataframe from the list, then drop the last row that doesn't have the value
df_concat['next_pc'] = next_pc_list
df_concat.dropna(inplace=True)

In [65]:
X = df_concat[['time_of_day', 'num_sensors','temperature']]
y = df_concat['power_consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [66]:
regr = DecisionTreeRegressor(max_depth=4)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.16374583369810392
MAPE :82.83403131201938%


In [67]:
regr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print_metrics(y_test, y_pred)

R2 Score: 0.06260694339668238
MAPE :82.56284903847269%


### Testing results

<table class="tg">
    <caption>Test results (1 hour)</caption>
<thead>
  <tr>
    <th class="tg-0pky">Dataset</th>
    <th class="tg-0pky">Number of samples</th>
    <th class="tg-0pky">R2 score</th>
    <th class="tg-0pky">Mean absolute percentage error</th>
    <th class="tg-0pky">Algorithm</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">Spring 2009 (7)</td>
    <td class="tg-0pky">2177</td>
    <td class="tg-0pky">0.23</td>
    <td class="tg-0lax">39.15%</td>
    <td class="tg-0lax">Decision Tree regressor</td>
  </tr>
  <tr>
    <td class="tg-0pky">Summer 2009 (8)</td>
    <td class="tg-0pky">1354</td>
    <td class="tg-0pky">0.442</td>
    <td class="tg-0lax">43.77%</td>
    <td class="tg-0lax">Decision tree regressor</td>
  </tr>
  <tr>
    <td class="tg-0pky">Merged dataset</td>
    <td class="tg-0pky">3531</td>
    <td class="tg-0pky">0.266</td>
    <td class="tg-0lax">49.47%</td>
    <td class="tg-0lax">Decision Tree regressor</td>
  </tr>
</tbody>
</table>


<table class="tg">
    <caption>Test results (15 minutes)</caption>
<thead>
  <tr>
    <th class="tg-0pky">Dataset</th>
    <th class="tg-0pky">Number of samples</th>
    <th class="tg-0pky">R2 score</th>
    <th class="tg-0pky">Mean absolute percentage error</th>
    <th class="tg-0pky">Algorithm</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">Spring 2009 (7)</td>
    <td class="tg-0pky">8676</td>
    <td class="tg-0pky">0.318</td>
    <td class="tg-0lax">73.83%</td>
    <td class="tg-0lax">Decision Tree regressor</td>
  </tr>
  <tr>
    <td class="tg-0pky">Summer 2009 (8)</td>
    <td class="tg-0pky">5047</td>
    <td class="tg-0pky">0.268</td>
    <td class="tg-0lax">50.31%</td>
    <td class="tg-0lax">Decision tree regressor</td>
  </tr>
  <tr>
    <td class="tg-0pky">Merged dataset</td>
    <td class="tg-0pky">13723</td>
    <td class="tg-0pky">0.164</td>
    <td class="tg-0lax">82.83%</td>
    <td class="tg-0lax">Decision tree regressor</td>
  </tr>
</tbody>
</table>